# About

"On Flying Backwards: Preventing Run-away of Small, Low-speed, Fixed-wing UAVs in Strong Winds"

This CoLab script contains basic formulations drived from the paper.

Note: Also, PX4 NPFG library functions are implemented as well for comparison.

# Environment Setup

In [ ]:
!pip install numpy
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import unittest

## Constants

In [ ]:
NPFG_EPSILON = 1.0e-6 # small number *bigger than machine epsilon

# Basic Formulations

## Bearing Feasibility

In [ ]:
def saturate(value, low, high):
  """ Saturates the value within the [low, high] boundary, without calculating the absolute value """
  return np.clip(value, low, high)

def saturate_bearing_PI_2(bearing_to_wind_rad):
  """ Saturates the absolute value of the bearing (in radians) within (0, PI/2) range """
  return np.clip(np.abs(bearing_to_wind_rad), 0, np.pi/2)

def bearing_feasibility_v1(bearing_to_wind_rad, wind_factor, DEBUG = False):
  """ Implementation of simple bearing feasibility function defined in Equation 3 """
  bearing_saturated = saturate_bearing_PI_2(bearing_to_wind_rad)
  # Note: This may return imaginary number (sqrt with negative number)
  numerator = np.sqrt(1 - np.square(wind_factor * np.sin(bearing_saturated)))
  denominator = np.cos(bearing_saturated)
  raw_feasibility = numerator / denominator
  
  if (DEBUG): print('Saturated Bearing: {}, Numerator: {}, Denominator: {}, Unprocessed feasibility: {}'.format(bearing_saturated, numerator, denominator, raw_feasibility))
  
  # Filter out the feasiblity value
  if np.isnan(raw_feasibility):
    return 0
  else:
    return np.clip(raw_feasibility, 0, 1)

def get_wind_factor_buffer_bounds(bearing_to_wind_rad, bearing_to_wind_cutoff_rad, wind_factor_buffer):
  """ Implementation of Equation 5 & 6, where wind factor buffer zone bounds are calculated (affected by bearing setpoint the most) """
  bearing_saturated = saturate_bearing_PI_2(bearing_to_wind_rad)
  
  if (bearing_saturated > bearing_to_wind_cutoff_rad):
    # Below the cutoff
    wind_factor_upper_inf = 1/np.sin(bearing_to_wind_cutoff_rad)
    wind_factor_slope = np.cos(bearing_to_wind_cutoff_rad) / np.square(np.sin(bearing_to_wind_cutoff_rad))
    wind_factor_lower_inf = (1/np.sin(bearing_to_wind_cutoff_rad) - 2) * wind_factor_buffer + 1

    wind_factor_upper = wind_factor_upper_inf + (bearing_to_wind_cutoff_rad - bearing_to_wind_rad) * wind_factor_slope
    wind_factor_lower = wind_factor_lower_inf + (bearing_to_wind_cutoff_rad - bearing_to_wind_rad) * wind_factor_slope * wind_factor_buffer
    return (wind_factor_lower, wind_factor_upper)
  else:
    # Above the cutoff
    wind_factor_upper = 1/np.sin(bearing_to_wind_rad)
    wind_factor_lower = (1/np.sin(bearing_to_wind_rad) - 2) * wind_factor_buffer + 1
    return (wind_factor_lower, wind_factor_upper)

def bearing_feasibility_v2(bearing_to_wind_rad, wind_factor, bearing_to_wind_cutoff_rad, wind_factor_buffer, DEBUG = False):
  """ Implementation of buffer-zone incorporated bearing feasibility function defined in Equation 3 """
  wind_factor_lower, wind_factor_upper = get_wind_factor_buffer_bounds(bearing_to_wind_rad, bearing_to_wind_cutoff_rad, wind_factor_buffer)

  if (DEBUG): print('Wind Factor bounds calculated: [{}, {}]'.format(wind_factor_lower, wind_factor_upper))

  if (wind_factor > wind_factor_upper):
    # Wind factor above the bound, infeasible bearing!
    return 0
  elif (wind_factor > wind_factor_lower):
    # Wind factor within the bound
    wind_factor_proportion = (wind_factor - wind_factor_lower) / (wind_factor_upper - wind_factor_lower)
    # Note: In the literature, it uses native saturation to (0, 1), range then multiplies by PI/2, but here I just directly re-use
    # the bearing saturation logic, by multiplying PI/2 before handing over to the function
    saturated_interpolated_angle = saturate_bearing_PI_2(wind_factor_proportion * (np.pi/2))
    feasibility_calculated = np.square(np.cos(saturated_interpolated_angle))
    if (DEBUG): print('Wind Factor within the bound! Proportion: {}, Saturated Angle: {}, Calculated feasibility: {}'.format(wind_factor_proportion, saturated_interpolated_angle, feasibility_calculated))
    return feasibility_calculated
  else:
    # Wind factor below the lower bound
    return 1

def bearing_feasibility_PX4(bearing_to_wind_rad, wind_speed, airspeed, airspeed_buffer = 1.5, DEBUG = False):
  """ Implementation of PX4 NPFG library's bearingFeasibility function """
  assert (wind_speed >= 0), "Wind speed must be positive number!"
  assert (airspeed >= 0), "Air speed must be positive number!"
  wind_cross_bearing_var = abs(wind_speed * np.sin(bearing_to_wind_rad))
  if (abs(bearing_to_wind_rad) > np.pi / 2):
    # If bearing is more than PI/2 beyond the wind velocity vector, override the crossed value
    wind_cross_bearing_var = wind_speed
  
  # NOTE: It is important that we saturate without taking the absolute value, as this logic
  # is different from the paper (`bearing_feasibility_v1`), and is based on speed magnitude
  # And for example, to account for a case where airspeed is too low, and feasibility goes to 0,
  # The saturate function should do what it's supposed to do only: Saturate the RAW value.
  sine_val = np.sin(saturate((airspeed - wind_cross_bearing_var) / airspeed_buffer, 0.0, 1.0) * np.pi / 2)

  if (DEBUG): print('Bearing: {}, Wind Speed: {}, Air Speed: {}, Wind cross bearing variable: {}, Sine value: {}'.format(bearing_to_wind_rad, wind_speed, airspeed, wind_cross_bearing_var, sine_val))
  return np.square(sine_val)


### Bearing Feasbilbility Test

In [ ]:
class TestBearingFeasiblity(unittest.TestCase):
  def test_saturate_bearing(self):
    """ Test Bearing saturation logic (implied in Eq.3) """
    input_bearings = [-np.pi, -np.pi*2/3, 0.01, np.pi/2, np.pi*4/5]
    output_saturated_bearings = [np.pi/2, np.pi/2, 0.01, np.pi/2, np.pi/2]
    for input_bearing, output_expected in zip(input_bearings, output_saturated_bearings):
      self.assertEqual(saturate_bearing_PI_2(input_bearing), output_expected)
  
  def test_bearing_feasibility_v1(self):
    """ Test Bearing Feasibility function without buffer zone """
    # Note: It will only test range bearing [0, PI] and wind factor [0, 2]
    DEBUG_ENABLE = False
    
    # >> Wind below aircraft's nominal airpseed (wind factor < 1)
    # Case 1: Wanting to fly with the wind (bearing = 0), Wind is minimal
    self.assertEqual(bearing_feasibility_v1(0, 0.01, DEBUG_ENABLE), 1)
    # Case 2: Wanting to fly against the wind (bearing = PI), Wind is close to airspeed
    self.assertEqual(bearing_feasibility_v1(np.pi, 0.98, DEBUG_ENABLE), 1)

    # >> Wind exactly equalling aircraft's nominal airpspeed (wind factor == 1)
    # Case 1: Wanting to fly with the wind
    self.assertEqual(bearing_feasibility_v1(0, 1.0, DEBUG_ENABLE), 1)
    # Case 2: Wanting to fly 45 deg skewed from the wind (maximum feasible bearing)
    self.assertEqual(bearing_feasibility_v1(np.pi/4, 1.0, DEBUG_ENABLE), 1)
    # Case 3: Wanting to fly 90 deg skewed from the wind (impossible)
    self.assertEqual(bearing_feasibility_v1(np.pi/2, 1.0, DEBUG_ENABLE), 0)
    # Case 4: Wanting to fly against the wind (impossible)
    self.assertEqual(bearing_feasibility_v1(np.pi, 1.0, DEBUG_ENABLE), 0)

    # >> Wind above aircraft's nominal airspeed (wind factor > 1)
    # Case 1: Wanting to fly with the wind
    self.assertEqual(bearing_feasibility_v1(0, 2.0, DEBUG_ENABLE), 1)
    # Case 2: Wanting to fly 30 deg skewed from the wind, with wind factor as sqrt(3) (maximum feasible bearing)
    # Since the feasibility will be a value between 0 and 1 (as it is an approximation), we assert that it's not 0.
    self.assertIsNot(bearing_feasibility_v1(np.pi/6, np.sqrt(3), DEBUG_ENABLE), 0)
    # Case 3: Wanting to fly 90 deg skewed from the wind, with wind factor 2 (impossible)
    # This is the case where the (wind factor * sin (bearing) > 1), resulting in invalid sqrt calculation.
    self.assertEqual(bearing_feasibility_v1(np.pi/2, 2.0, DEBUG_ENABLE), 0)

  def test_wind_factor_buffer_bounds(self):
    """ Test Wind factor buffer calculation """
    # >> Bearing target above cutoff
    
    # >> Bearing target below cutoff
    return

  def test_bearing_feasibility_v2(self):
    """ Test Wind buffer-incorporated bearing feasibility function """
    return

  def test_bearing_feasibility_PX4(self):
    """ Test PX4 NPFG library's Bearing Feasibility function """
    # Arbitrary airspeed of the vehicle (e.g. Hobbywing Z-84 would fly at this speed)
    AIRSPEED = 10
    DEBUG_ENABLE = True

    # We conceptually have a dial on the 'wind factor', and use the resulting wind speed as an agrument
    # >> Wind below the vehicle airspeed
    for wind_factor in [0.0, 0.5]:
      WINDSPEED = AIRSPEED * wind_factor
      # Case 1: Bearing going with the wind
      self.assertEqual(bearing_feasibility_PX4(0.0, WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 1)

      # Case 3: Bearing perpendicular to the wind
      self.assertEqual(bearing_feasibility_PX4(np.pi/2, WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 1)

      # Case 4: Bearing going fully against the wind
      self.assertEqual(bearing_feasibility_PX4(np.pi, WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 1)
    
    # >> Wind above or equal to vehicle airspeed
    for wind_factor in [1.0, 1.5, 2.0]:
      WINDSPEED = AIRSPEED * wind_factor
      # Case 1: Bearing going with the wind
      self.assertEqual(bearing_feasibility_PX4(0.0, WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 1)

      # Case 2: Bearing at half of sub-PI/2 angle around physical feasibility boundary
      # Feasibility should be between 0 and 1, then it shouldn't be plain 0!
      self.assertIsNot(bearing_feasibility_PX4(np.arcsin(1/wind_factor)/2, WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 0)

      # Case 2: Bearing at sub-PI/2 angle around physical feasibility boundary
      # Feasibility should be between 0 and 1, then it shouldn't be plain 0!
      # Note, the function returns 0, but not sure why 'assertIsNot' doesn't trigger :(
      print(bearing_feasibility_PX4(np.arcsin(1/wind_factor), WINDSPEED, AIRSPEED))
      self.assertIsNot(bearing_feasibility_PX4(np.arcsin(1/wind_factor), WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 0)

      # Case 3: Bearing perpendicular to the wind
      self.assertEqual(bearing_feasibility_PX4(np.pi/2, WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 0)

      # Case 4: Bearing going fully against the wind
      self.assertEqual(bearing_feasibility_PX4(np.pi, WINDSPEED, AIRSPEED, DEBUG = DEBUG_ENABLE), 0)

# Run the unit test
# https://medium.com/@vladbezden/using-python-unittest-in-ipython-or-jupyter-732448724e31
unittest.main(argv=['first-arg-is-ignored'], exit=False)

.<ipython-input-17-f686ba1d29ee>:13: RuntimeWarning: invalid value encountered in sqrt
  numerator = np.sqrt(1 - np.square(wind_factor * np.sin(bearing_saturated)))
....

Bearing: 0.0, Wind Speed: 0.0, Air Speed: 10, Wind cross bearing variable: 0.0, Sine value: 1.0
Bearing: 1.5707963267948966, Wind Speed: 0.0, Air Speed: 10, Wind cross bearing variable: 0.0, Sine value: 1.0
Bearing: 3.141592653589793, Wind Speed: 0.0, Air Speed: 10, Wind cross bearing variable: 0.0, Sine value: 1.0
Bearing: 0.0, Wind Speed: 5.0, Air Speed: 10, Wind cross bearing variable: 0.0, Sine value: 1.0
Bearing: 1.5707963267948966, Wind Speed: 5.0, Air Speed: 10, Wind cross bearing variable: 5.0, Sine value: 1.0
Bearing: 3.141592653589793, Wind Speed: 5.0, Air Speed: 10, Wind cross bearing variable: 5.0, Sine value: 1.0
Bearing: 0.0, Wind Speed: 10.0, Air Speed: 10, Wind cross bearing variable: 0.0, Sine value: 1.0
Bearing: 0.7853981633974483, Wind Speed: 10.0, Air Speed: 10, Wind cross bearing variable: 7.071067811865475, Sine value: 1.0
0.0
Bearing: 1.5707963267948966, Wind Speed: 10.0, Air Speed: 10, Wind cross bearing variable: 10.0, Sine value: 0.0
Bearing: 1.570796326794896


----------------------------------------------------------------------
Ran 5 tests in 0.025s

OK


## Other functions

In [ ]:
def windFactor(airspeed, wind_speed):
  """ Calculates approximation of a Wind Factor """
  if (wind_speed > airspeed):
    return 2.0
  else:
    return 2 * (1 - np.sqrt(1 - min (1.0, wind_speed/airspeed)))

def trackErrorBound(ground_speed, time_constant):
  """ Track error upper limit at which quadratic look-ahead angle transition happens """
  if (ground_speed > 1.0):
    return ground_speed * time_constant
  else:
    return 0.5 * time_constant * (np.square(ground_speed) + 1.0)

def periodLowerBound(air_turn_rate, wind_factor, feasability_on_track, roll_time_constant, damping):
  """ Lower bound for the period for the controller """
  period_min = np.pi * roll_time_constant / damping

  if ((air_turn_rate * wind_factor < NPFG_EPSILON) or (damping < 0.5)):
    return period_min
  else:
    period_windy_curved_damped = 4.0 * np.pi * roll_time_constant * damping
    return np.interp(feasability_on_track, [0.0, 1.0], [period_min, period_windy_curved_damped])

### Other Functions Test

In [ ]:
class TestOtherFunctionsFeasiblity(unittest.TestCase):
  def test_windFactor(self):
    # No wind: Wind Factor 0
    self.assertEqual(windFactor(10.0, 0.0), 0.0)
    # No wind: Wind Factor 0.5
    self.assertTrue(windFactor(10.0, 5.0) > 0.4 and windFactor(10.0, 5.0) < 0.6)
    # No wind: Wind Factor 1.0
    self.assertEqual(windFactor(10.0, 10.0), 2.0)
    # No wind: Wind Factor 1.5
    self.assertEqual(windFactor(10.0, 15.0), 2.0)

# https://docs.python.org/3/library/unittest.html#organizing-test-code
# def suite():
#   suite = unittest.TestSuite()
#   suite.addTest(TestOtherFunctionsFeasiblity('test_windFactor'))
#   return suite
suite = unittest.TestSuite()
suite.addTest(TestOtherFunctionsFeasiblity('test_windFactor'))
runner = unittest.TextTestRunner()
runner.run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

### Other Functions Graph